In [1]:
import multiprocessing
from GCForest import gcForest
import pandas as pd
import numpy as np
import pickle 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.preprocessing import Imputer
import warnings
warnings.filterwarnings("ignore")
#用来计算程序运行时间
import datetime
starttime = datetime.datetime.now()

In [2]:
#读取Y
all_df_y_2c = pickle.load(open("./dump_file/all_df_y_2c","rb"))
print("Y.shape:",all_df_y_2c.shape)
all_df_y = pickle.load(open("./dump_file/all_df_y","rb"))
all_df_y['2cArousal'] = 0
all_df_y['2cArousal'][all_df_y['valence'] >= 5] = 1
all_df_y['2cValence'] = 0
all_df_y['2cValence'][all_df_y['valence'] >= 5] = 1
print(all_df_y.head(5))
#读取32个通道的EEG数据，每个通道包含32×40=1280个信号样本（人次×每人次40实验）
#每个样本向量大小为8064点（63s*128Hz）
for eegCH in range(1,33,1):
    file_path = "./dump_file/CH{}_df_EEG_x".format(eegCH)
    locals()['CH{}_df_EEG_x'.format(eegCH)] = pickle.load(open(file_path,"rb"))
    #file_path = "./dump_file/CH{}eegfft_df".format(eegCH)
    #locals()["CH{}eegfft_df".format(eegCH)] = pickle.load(open(file_path,"rb"))
'''
X = CH1_df_EEG_x
y = all_df_y[["2cArousal"]]
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.3,stratify=y,random_state=2018)   
xTrainIdx = X_tr.index
xTestIdx = X_te.index
'''

#y = all_df_y[['2cValence']]
y = all_df_y[["2cArousal"]]
xTrainIdx = pickle.load(open("./dump_file/xTrainIdx","rb"))
xTestIdx = pickle.load(open("./dump_file/xTestIdx","rb"))
y_tr = y.loc[xTrainIdx]
y_te = y.loc[xTestIdx]

Y.shape: (1280, 1)
       valence  arousal  dominance  liking  2cArousal  2cValence
s01_0     7.71     7.60       6.90    7.83          1          1
s01_1     8.10     7.31       7.28    8.47          1          1
s01_2     8.58     7.54       9.00    7.08          1          1
s01_3     4.94     6.01       6.12    8.06          0          0
s01_4     6.96     3.92       7.19    6.05          1          1


In [3]:
myWindowsSize = 256
myStrideSize = 64
gcf = gcForest(shape_1X=8064, window=myWindowsSize, stride=myStrideSize,tolerance=0.0,n_cascadeRF=1, 
               min_samples_mgs=0.1, min_samples_cascade=0.1,n_jobs=19)

In [4]:
#这部分是时域的
for eegCH in range(1,33,1):
    filePath = "./dump_file/CH{}mgsTrainVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
    locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
    filePath = "./dump_file/CH{}mgsTestVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
    locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))
#确认下有没有缺失值
for eegCH in range(1,33,1):
    print("CH{}".format(eegCH))
    print(locals()['CH{}mgsTrainVector'.format(eegCH)].max())
    print(locals()['CH{}mgsTestVector'.format(eegCH)].max())
X_tr_vector = locals()['CH{}mgsTrainVector'.format(1)]
X_te_vector = locals()['CH{}mgsTestVector'.format(1)]
#拼接MGS处理的各个通道向量---》cascade forest
for eegCH in range(2,33,1):
    nextTrainVector = locals()['CH{}mgsTrainVector'.format(eegCH)]
    nextTestVector = locals()['CH{}mgsTestVector'.format(eegCH)]
    X_tr_vector = np.concatenate([X_tr_vector,nextTrainVector],axis=1)
    X_te_vector = np.concatenate([X_te_vector,nextTestVector],axis=1)
print(X_tr_vector.shape)
print(X_te_vector.shape)

CH1
0.7393425225060231
0.7199696815754925
CH2
nan
0.6536154048130004
CH3
0.7065762832401695
0.6901042372509257
CH4
0.7592447489170969
0.73913380150607
CH5
0.7372107511569954
0.7152836932582054
CH6
0.7817325038959655
0.7566017581456896
CH7
0.7523832925298628
0.7172852793177056
CH8
0.7498968594974604
0.7358275513316536
CH9
0.7451251548465961
0.7341108012946533
CH10
0.7781864969061605
0.7360549667728011
CH11
0.7408645113106175
0.7187428198683746
CH12
0.7154402586019015
0.6833616404341655
CH13
0.7171294091231729
0.6866329571402064
CH14
nan
0.7237902500029013
CH15
0.7192575665721391
0.6990849122425941
CH16
0.7937012444975694
0.7221024195641648
CH17
0.7214214912547328
0.7061780458356609
CH18
0.7548790257524605
0.6964281089731403
CH19
0.7250106541513599
0.7053664431790935
CH20
0.7403599112657314
0.7297570844173796
CH21
0.7406040291015084
0.7047846483796527
CH22
0.7572788755484041
0.7159216635382176
CH23
0.7720978113544644
0.7374748442836325
CH24
0.7440319142430404
0.7044214071675966
CH25
0.71

In [5]:
#这部分是频域的
for eegCH in range(1,33,1):
    filePath = "./dump_file_V2/CH{}mgsTrainVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
    locals()['CH{}mgsTrainVector'.format(eegCH)] = pickle.load(open(filePath,"rb"))
    filePath = "./dump_file_V2/CH{}mgsTestVector_{}_{}".format(eegCH,myWindowsSize,myStrideSize)
    locals()['CH{}mgsTestVector'.format(eegCH)]= pickle.load(open(filePath,"rb"))
#确认下有没有缺失值
for eegCH in range(1,33,1):
    print("CH{}".format(eegCH))
    print(locals()['CH{}mgsTrainVector'.format(eegCH)].max())
    print(locals()['CH{}mgsTestVector'.format(eegCH)].max())
#在已经有时域数据的基础上拼接MGS处理的各个通道向量---》cascade forest
for eegCH in range(1,33,1):
    nextTrainVector = locals()['CH{}mgsTrainVector'.format(eegCH)]
    nextTestVector = locals()['CH{}mgsTestVector'.format(eegCH)]
    X_tr_vector = np.concatenate([X_tr_vector,nextTrainVector],axis=1)
    X_te_vector = np.concatenate([X_te_vector,nextTestVector],axis=1)
print(X_tr_vector.shape)
print(X_te_vector.shape)

CH1
0.7538653750648605
0.7247982468150902
CH2
0.7880665839840312
0.7469153545090329
CH3
0.8304334471142862
0.7638411005534081
CH4
0.7563080642347763
0.7409899067865713
CH5
nan
0.7208661504571021
CH6
0.8415424565444407
0.8210498016206313
CH7
0.7623340219500898
0.7558795740916715
CH8
0.7444661427772885
0.7101267715275024
CH9
0.7597522472649912
0.6798072173998975
CH10
0.7147573248822854
0.6858181406002213
CH11
0.7331547336416568
0.7046968376317139
CH12
nan
0.7018186753215888
CH13
0.7803058075206216
0.750904510611852
CH14
0.8645657316271181
0.7695517200539633
CH15
0.8177879999473631
0.7438874685660222
CH16
0.7370075288846778
0.7054490264972635
CH17
0.7621538211339776
0.7284614517362236
CH18
0.8081950899357735
0.7339725797657165
CH19
0.8330369805265404
0.6856578208415205
CH20
0.7054572369881791
0.6927299272806314
CH21
nan
0.7266690806692324
CH22
0.7051466506965284
0.6923686867597709
CH23
nan
0.695737812174922
CH24
0.769630948521307
0.752819008157012
CH25
0.7562776875217644
0.713587064247496

In [6]:
#如有缺失值，填充下
X_tr_vector_fillna= pd.DataFrame(X_tr_vector).fillna(0).values
X_te_vector_fillna= pd.DataFrame(X_te_vector).fillna(0).values

In [7]:
_ = gcf.cascade_forest(X_tr_vector_fillna, y_tr)

Adding/Training Layer, n_layer=1
Layer validation accuracy = 0.6611111111111111
Adding/Training Layer, n_layer=2
Layer validation accuracy = 0.6444444444444445


In [8]:
pred_proba = gcf.cascade_forest(X_te_vector_fillna)
tmp = np.mean(pred_proba, axis=0)
preds = np.argmax(tmp, axis=1)
print("ACC",accuracy_score(y_true=y_te, y_pred=preds))
print("F1",f1_score(y_true=y_te, y_pred=preds))
print("Recal",recall_score(y_true=y_te, y_pred=preds))
print("Precision",precision_score(y_true=y_te, y_pred=preds))

ACC 0.6927083333333334
F1 0.7713178294573644
Recal 0.9170506912442397
Precision 0.6655518394648829
